In [1]:
import re
import math
import sys
sys.path.append("../../src/")
import uncertainpy.gradual as grad
import time
from statistics import median
import random
random.seed(0)

In [2]:
# Parse a QBAF file to an adjacency matrix
def parse_qbaf(filename):
    num_nodes = 0
    with open(filename, 'r') as file:
        for line in file:
            match = re.match(r'(\w+)\((\d+), ([\d.]+)\)', line)
            if match:
                type, node_id1, weight = match.groups()
                if type == 'arg':
                    num_nodes += 1
            else:
                raise ValueError(f"can't parse this line: {line}")
        adj_matrix = [[0] * num_nodes for _ in range(num_nodes)]

    with open(filename, 'r') as file:
        for line in file:
            match = re.match(r'(\w+)\((\d+), ([\d.]+)\)', line)
            if match:
                type, node_id1, node_id2 = match.groups()
                if type == 'att':
                    adj_matrix[int(node_id1)][int(node_id2)] = -1
                elif type == 'sup':
                    adj_matrix[int(node_id1)][int(node_id2)] = 1
            else:
                raise ValueError(f"can't parse this line: {line}")
    return adj_matrix

In [3]:
# Convert the adjacency matrix to a python dictionary
def adjacency_matrix_to_dict(adj_matrix):
    graph_dict = {}

    for i, row in enumerate(adj_matrix):
        neighbors = set(j for j, weight in enumerate(row) if weight != 0)
        graph_dict[i] = neighbors

    return graph_dict

In [4]:
# Check if a node contains a loop to itself
def find_cycles(graph, start, current, visited, path, cycles):
    visited[current] = True
    path.append(current)
    for neighbor in graph[current]:
        if neighbor == start and len(path) > 0:
            # find a cycle
            cycles.append(path.copy()+[neighbor])
        elif not visited[neighbor]:
            find_cycles(graph, start, neighbor, visited, path, cycles)
    # backtracking
    visited[current] = False
    path.pop()

# Check all the nodes in a QBAF if they contain loops to themselves, if there is any, then the QBAF is cyclic.
def find_all_cycles(graph):
    cycles = []
    num_nodes = len(graph)
    visited = [False] * num_nodes
    for node in range(num_nodes):
        find_cycles(graph, node, node, visited, [], cycles)
    return cycles

# Compute the polarity of a cycle (cycles are also paths).
def compute_cycle_polarity(cycles, adj_matrix):
    cycles_polarity_dict = {}

    for cycle in cycles:
        cycles_polarity_dict[cycle[0]] = 1 # initialise

    for cycle in cycles:
        if cycles_polarity_dict[cycle[0]] != -1:
            temp = 1
            for i in range(len(cycle)-1):
                temp *= adj_matrix[cycle[i]][cycle[i+1]]
            if temp == -1:
                cycles_polarity_dict[cycle[0]] = -1
                continue
    return cycles_polarity_dict

In [5]:
# Compute all the paths between two arguments
def find_all_paths_between_two_args(graph, start, end, path=[]):
    path = path + [start]
    if start == end:
        return [path]
    if start not in graph:
        return []
    paths = []
    for node in graph[start]:
        if node not in path:
            new_paths = find_all_paths_between_two_args(graph, node, end, path)
            for new_path in new_paths:
                paths.append(new_path)
    return paths

def find_all_paths_between_two_args_complete(graph, start, end, path=[]):
    if start == end:
        return []
    else:
        paths = find_all_paths_between_two_args(graph, start, end)
        return paths

In [6]:
# Compute the (direct or indirect) polarity from one node to another, return an integer
def compute_polarity_between_two_args(graph, start_node, end_node, cycles_polarity, adj_matrix):
    paths = find_all_paths_between_two_args_complete(graph, start_node, end_node)
    paths_count = len(paths)
    # print(paths_count)
    # print(paths)

    # case1: self to self:
    if start_node == end_node:
        return 1 # positive

    # case2: no path
    if paths_count == 0:
        return -2 # neutral

    # case3: 1 path or more than 1
    elif paths_count >= 1:
        node_in_paths = {element for sublist in paths for element in sublist}
        for node in node_in_paths:
            if node in cycles_polarity and cycles_polarity[node] == -1:
                return 0 # contains negative cycles, thus unknown

        # record the polarity for each path
        recodr_polarity = [1] * paths_count
        for j in range(paths_count):
            for i in range(len(paths[j])-1):
                recodr_polarity[j] *= adj_matrix[paths[j][i]][paths[j][i+1]]

        if all(x == 1 for x in recodr_polarity):
            pol = 1 # positive: if all the paths are positive
        elif all(x == -1 for x in recodr_polarity):
            pol = -1 # negative: if all the paths are negative
        else:
            pol = 0 # unknown: some paths positive while some negative

        return pol

In [7]:
# Compute the polarity for all the nodes to the topic argument, return a vector
def compute_polarity_vector(graph, topic_arg, cycles_polarity, adj_matrix):
    num_nodes = len(graph)
    polarity_vector = [-2] * num_nodes
    for i in range(num_nodes):
        polarity_vector[i] = compute_polarity_between_two_args(graph, i, topic_arg, cycles_polarity, adj_matrix)
    return polarity_vector

In [8]:
# Compute priority (not polarity) between two arguments
def compute_priority_between_two_args(graph, start, end):
    if start == end:
        return 3 # itself
    paths = find_all_paths_between_two_args(graph, start, end)
    if len(paths) == 0:
        return 0 # disconnected
    min_length = min(len(path) for path in paths) - 1
    priority = 1 / min_length # single/multi-path connected
    return priority

In [9]:
# Compute the priority for all arguments to the topic argument
def compute_priority_vector(graph, topic_arg):
    num_nodes = len(graph)
    priority_vector = [0] * num_nodes
    for i in range(num_nodes):
        priority_vector[i] = compute_priority_between_two_args(graph, i, topic_arg)
    return priority_vector

In [10]:
def compute_sparsity(bs1, bs2):
    count = 0
    for item in bs1:
        if bs1[item]!=bs2[item]:
            count += 1
    return count

In [11]:
def compute_potential_cause_cyclic_sparsity(filename, topic_arg, desired_strength, polarity, priority, model):

    model.approximator = grad.algorithms.RK4(model)
    model.BAG = grad.BAG(filename)
    model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)

    pre_strength = -1

    # print(f"desired_strength:{desired_strength}")
    initial_strength = model.BAG.arguments[str(topic_arg)].strength
    cur_strength = initial_strength
    print(f"cur_strength:{initial_strength}")

    # rank the priority list
    sorted_priority = sorted(priority, reverse=True)
    indices_sorted_priority = [index for index, value in sorted(enumerate(priority), key=lambda x: x[1], reverse=True)]


    # traverse all the argument from the highest priority to the lowest
    for i in indices_sorted_priority:
        if pre_strength == cur_strength:
            break
        if (desired_strength - cur_strength) * (desired_strength - initial_strength) > 0: # initial strength and current strength still at the same side

            # increase positive and decrease negative
            if (desired_strength - cur_strength) * polarity[i] > 0:
                model.BAG.arguments[str(i)].reset_initial_weight(1.0)
            elif (desired_strength - cur_strength) * polarity[i] < 0:
                model.BAG.arguments[str(i)].reset_initial_weight(0.0)

            # re-compute the strength
            print(f"desired_strength:{desired_strength}")
            pre_strength = cur_strength
            model.solve(delta=10e-2, epsilon=10e-4, verbose=False, generate_plot=False)
            cur_strength = model.BAG.arguments[str(topic_arg)].strength
            print(f"cur_strength:{cur_strength}")

            # print current bs
            current_bs_dict = {}
            for arg in model.BAG.arguments.values():
                current_bs_dict[arg.get_name()] = arg.get_initial_weight()
            print(f"current base scores: {current_bs_dict}")

    # print current bs
    current_bs_dict = {}
    for arg in model.BAG.arguments.values():
        current_bs_dict[arg.get_name()] = arg.get_initial_weight()
    print(f"current base scores: {current_bs_dict}")

    # compute validity
    validity = False
    if (desired_strength - cur_strength) * (desired_strength - initial_strength) < 0 or desired_strength == cur_strength:
        validity = True

    return current_bs_dict, validity

In [12]:
def main():

    start_time = time.time() # record the current time


    # set parameters
    filename = f'../../bags/cyclic_0.bag'
    topic_arg = 0
    desired_strength = 0.5


    # obtain origin_base_score_dict
    bag = grad.BAG(filename)
    origin_base_score_dict = {}
    for arg in bag.arguments.values():
        origin_base_score_dict[arg.name] = arg.get_initial_weight()
    # print(f"origin_base_score_dict:{origin_base_score_dict}")


    # compute cycles, polarity, priority
    adj_matrix = parse_qbaf(filename)
    graph_dict = adjacency_matrix_to_dict(adj_matrix)
    cycles = find_all_cycles(graph_dict)
    cycles_polarity_dict = compute_cycle_polarity(cycles, adj_matrix)
    polarity = compute_polarity_vector(graph_dict, topic_arg, cycles_polarity_dict, adj_matrix)
    # priority = [1] * len(origin_base_score_dict)
    priority = compute_priority_vector(graph_dict, topic_arg)
    # print(f"cycle polarity:{cycles_polarity_dict}")
    # print(f"polarity:{polarity}")
    # print(f"priority:{priority}")

    # compute a potential cause
    model = grad.semantics.QuadraticEnergyModel()
    potential_cause_dict, validity = compute_potential_cause_cyclic_sparsity(filename, topic_arg, desired_strength, polarity, priority, model)

    # Compute sparsity
    sparsity = compute_sparsity(origin_base_score_dict, potential_cause_dict)

    end_time = time.time()
    runtime = end_time - start_time
    print(f"Runtime: {runtime}")

    print(f"================================================")
    print(f"Summary Results:")
    print(f"Validity:{validity}")
    print(f"Sparsity:{sparsity}")
    print(f"Runtime:{runtime}")

if __name__ == "__main__":
    main()

cur_strength:0.25
desired_strength:0.5
cur_strength:0.9994400569917434
current base scores: {'0': 1.0, '1': 0.49, '2': 0.2, '3': 0.08, '4': 0.27, '5': 0.19, '6': 0.74, '7': 0.88, '8': 0.14, '9': 0.15, '10': 0.13, '11': 0.93, '12': 0.11, '13': 0.71, '14': 0.48, '15': 0.22, '16': 0.8, '17': 0.46, '18': 0.55, '19': 0.86, '20': 0.69, '21': 0.47, '22': 0.58, '23': 0.73, '24': 0.21, '25': 0.37, '26': 0.29, '27': 0.03, '28': 0.62, '29': 0.96, '30': 0.94, '31': 0.74, '32': 0.17, '33': 0.76, '34': 0.57, '35': 0.67, '36': 0.69, '37': 0.95, '38': 0.68, '39': 0.44, '40': 0.95, '41': 0.59, '42': 0.03, '43': 0.3, '44': 0.41, '45': 0.42, '46': 0.43, '47': 0.17, '48': 0.54, '49': 0.86, '50': 0.19, '51': 0.07, '52': 0.47, '53': 0.68, '54': 0.57, '55': 0.23, '56': 0.71, '57': 0.39, '58': 0.08, '59': 0.1, '60': 0.39, '61': 0.92, '62': 0.61, '63': 0.56, '64': 0.39, '65': 0.93, '66': 0.92, '67': 0.24, '68': 0.1, '69': 0.29, '70': 0.66, '71': 0.84, '72': 0.86, '73': 0.53, '74': 0.24, '75': 0.32, '76': 0.23,